# Demo

In [1]:
# !pip install tai-chi-engine

## Probable other dependencies
transformers==4.12.3

In [1]:
from forgebox.imports import *
from tai_chi_engine import TaiChiEngine

## Demo tasks

> Load all the code above in one shot, the demo starts here

In [2]:
def df_creator_image_folder(path: Path) -> pd.DataFrame:
    """
    Create a dataframe ,
    Which list all the image path under a system folder
    """
    path = Path(path)
    files = []
    formats = ["jpg", "jpeg", "png"]
    for fmt in formats:
        files.extend(path.rglob(f"*.{fmt.lower()}"))
        files.extend(path.rglob(f"*.{fmt.upper()}"))
    return pd.DataFrame({"path": files}).sample(frac=1.).reset_index(drop=True)

### Choose dataset

In [3]:
# BEAR_DATASET = HOME/"Downloads"/"bear_dataset"
DATA = Path("/GCI/data")
BEAR_DATASET = DATA/"bear_dataset"
ROTTEN_TOMATOES = DATA/"rttmt"
NETFLIX = DATA/"nf"

Choose one of the following to run 

#### Netflix 📺

In [4]:
base_df = pd.read_csv(NETFLIX/"netflix_titles.csv")
base_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


#### The bear 🐻

In [4]:
base_df = df_creator_image_folder(BEAR_DATASET)
base_df

,path
0,/GCI/data/bear_dataset/black/00000008.jpg
1,/GCI/data/bear_dataset/teddys/00000056.jpg
2,/GCI/data/bear_dataset/grizzly/00000116.jpg
3,/GCI/data/bear_dataset/black/00000059.jpg
4,/GCI/data/bear_dataset/black/00000136.JPG
...,...
517,/GCI/data/bear_dataset/black/00000183.jpg
518,/GCI/data/bear_dataset/grizzly/00000184.jpg
519,/GCI/data/bear_dataset/teddys/00000085.jpg
520,/GCI/data/bear_dataset/grizzly/00000068.jpg


#### The rotten tomatoes 🍅 🎬

In [4]:
# the rotten tomatoes dataset, we are not using every line

base_df = pd.read_csv(ROTTEN_TOMATOES/'critic_reviews.csv', nrows=200000)
base_df = base_df[~base_df['review_score'].isna()].reset_index(drop=True)
base_df = base_df[~base_df['review_content'].isna()].reset_index(drop=True)
base_df = base_df[~base_df['critic_name'].isna()].reset_index(drop=True)

base_df = base_df[base_df['review_score'].apply(lambda x: "/" in x)].reset_index(drop=True)

base_df['review_score'] = base_df['review_score'].apply(eval)

base_df

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,0.700,2010-02-09,Whether audiences will get behind The Lightnin...
1,m/0814255,Nick Schager,False,Slant Magazine,Rotten,0.250,2010-02-10,Harry Potter knockoffs don't come more transpa...
2,m/0814255,Bill Goodykoontz,True,Arizona Republic,Fresh,0.700,2010-02-10,"Percy Jackson isn't a great movie, but it's a ..."
3,m/0814255,Jim Schembri,True,The Age (Australia),Fresh,0.600,2010-02-10,"Crammed with dragons, set-destroying fights an..."
4,m/0814255,Mark Adams,False,Daily Mirror (UK),Fresh,0.800,2010-02-10,"This action-packed fantasy adventure, based on..."
...,...,...,...,...,...,...,...,...
108237,m/bottle_shock,Phil Villarreal,False,Arizona Daily Star,Rotten,0.500,2008-08-29,"It might have worked better as a documentary, ..."
108238,m/bottle_shock,Todd Gilchrist,False,IGN Movies,Rotten,0.400,2008-08-29,Bottle Shock feels more like an excuse to exer...
108239,m/bottle_shock,Austin Kennedy,False,Sin Magazine,Rotten,0.625,2008-09-02,"I was slightly involved towards the end, but t..."
108240,m/bottle_shock,Sean P. Means,False,Salt Lake Tribune,Rotten,0.500,2008-09-05,"Flat, musty and with a hint of flopsweat."


### Start of the pipeline

Initiate the ```phase``` to track the configuration

In [5]:
# PROJECT = Path("./project")
# PROJECT = Path("./project/image_regression")
# PROJECT = Path("./project/rotten1")
# PROJECT = Path("./project/rotten_text")
PROJECT = Path("./project2/netflix")
# PROJECT = Path("./project0")
# PROJECT = Path("./project3")

engine = TaiChiEngine(base_df, project = PROJECT)

In [6]:
engine()

<IPython.core.display.Javascript object>

Output()

In [7]:
engine.progress["model"].optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 1e-05
    weight_decay: 0.0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0.0
)